# Exercise setup and information

Click on the link below to begin the exercise:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/anarekuli011/Euroavia/blob/main/Euroavia_Exercise.ipynb)

In [ ]:
"""                               PLEASE READ THESE POINTS BEFORE YOU START WITH THE EXERCISE                                         """


"""1. Check if you have all the files from the repository cloned correctly into Google Colab. If any files are missing retry the cloning cell, or add them manually."""

"""2. The sections are marked with 'Enter your code here'. But you still need to go through each section to understand the task."""

"""3. You have to press run on each cell as you go through the notebook. You can do this by pressing the play button in the left corner of each cell or ctr+shift+enter."""

"""4. Be sure to read all the comments in each cell."""

"""5. If you have any questions, or problems feel free to reach out! This isn't an exam 😁."""

"""                                                    GOOD LUCK!                                                                     """

"""                               PLEASE READ THESE POINTS BEFORE YOU START WITH THE EXERCISE                                         """

In [ ]:
""" In this exercise we will take you through a project that is currently ongoing at ADB Safegate """

""" You will experience with real examples, what it's like to be a part of Airside 4.0 that is currently transforming airfield ecosystem to join the Digital and Information Age. """

""" Lets start by running this code below to clone the git repository containing all the files you will need. """


#If you want to retry cloning, first go to 'runtime'->'disconnect and delete runtime', and then run this cell again
!git clone https://github.com/anarekuli011/Euroavia_Files.git

In [ ]:
"""

    Run this cell to import all the libraries you will need to complete the exercise.
    You can import other libraries if you would like to solve this exercise with a different method.

"""

import sys
sys.path.append('/content/Euroavia_Files')
import os
os.chdir('/content/Euroavia_Files')


import Functions as fn

In [ ]:
"""

    Run this cell to get a quick look at how the technology in the airfield industry has progressed along side the advancements in technology.

    Here is the timeline in which those technologies have been developed:
    Airside 1.0 (1910) --> Airside 2.0 (1920) --> Airside 3.0 (1969) --> Airside 4.0 (2022)

    So the recent development in Airside 4.0 is a new game changing development, the biggest change to the industry since 1969.

"""
fn.show_img('Images/AIRSIDE 4.0 Story.png', 10, 6)

In [ ]:
"""

    Here you will get an idea of how ALCMS (Airfield Lighting Control and Monitoring Systems) are set up.

    You can see the architecture below highlights how the system goes from a high-level view "Airport Name" all the way down to each regulator.

"""

fn.show_img('Images/ALCMS Architecture.png')

In [ ]:
"""

    The goal of the current project (from which this exercise is taken) is to explore how this architecture can be mapped to a table format and stored on Azure Databricks.
    But also mapped to individual assets and lights, and their specific locations (in coordinates), as this data already exists in Databricks.

    Doing this has several benefits mainly - information transparency, both internal and external.
    As we can use this stored data to create:

    1. High-level views that beneficial for managers to see KPIs and identify focus points in development and operations.
    2. In-depth dashboard at Region level for internal use to analyze growth and operations.
    3. In-depth dashboards at Airport level for clients to more access to their ALCMS system's information.

"""


fn.show_img('Images/Connection to CXS.png', 10, 6)


In [ ]:
"""
    Before excercise 1, lets start with looking at how the first goal of this project was tackled:

    We want to to map the ALCMS architecture into a table format and store it on Azure Databricks.
    For the scope of this excercise we will be putting the transformed data into an excel sheet.

"""

In [ ]:
"""
    Imports
"""

import xml.etree.ElementTree as ET
import pandas as pd

In [ ]:
"""
    Now lets take a look at the XML file for Queenstown Airport, break down all the information one cell at a time.
"""
#XML File
xml_path = "NZQN - NewZealand - Queenstown.xml"

tree = ET.parse(xml_path)
root = tree.getroot()

#Print some high level information about the xml file.
print(f"Root tag: {root.tag}")
print("Root attributes:")
for k in sorted(root.attrib):
    print(f"  {k}: {root.attrib[k]}")
print(f"Number of direct children of root: {len(root)}")

In [ ]:
"""
    The previous cell gives you information about the ALCMS design, its the first step in identifying what is needed from the xml file.

    Run this cell to see a general visual representaion of this design.
"""

fn.show_img('Images/ALCMS Design.png', 5, 3)

In [ ]:
"""
    The path used from 'ALCMSServer->InternalALCMSDatabase->ALCMSDataBase' is where we can find the necessary information to map the architecture we previously discussed.
    So we can run the code below to store that information in a pandas dataframe, and also print its attributes to visualise it like we did with the root of the xml file.
"""

db = root.find("./ALCMSServer/InternalALCMSDataBase/ALCMSDataBase")
if db is None:
    raise ValueError("No <ALCMSDataBase> found under <ALCMSServer>/<InternalALCMSDataBase>.")

print("ALCMSDataBase attributes:", db.attrib)
print("Number of direct children:", len(list(db)))

print("\nDirect child tags under ALCMSDataBase (unique):")
print(sorted({child.tag for child in db}))

In [ ]:
"""
    You should recognize the different child tags under ALCMSDataBase from the image for the ALCMS architecture.
    Lets compare them again in this cell.
"""

print("\nDirect children (tag, attrib) - first 200:")
for child in list(db)[:200]:
    print(" -", child.tag, child.attrib)

fn.show_img('Images/ALCMS Architecture.png', 10, 6)

# Exercise 1




In [ ]:
"""
    Here are two functions that will help you find and append the information you need from the xml file.
    You don't need to change them, but they are being shown explicitly so that you can better understand what type of table we are aiming for.
"""

def get_alcms_database(root):
    server = root.find('ALCMSServer')
    if server is not None:
        internal_db = server.find('InternalALCMSDataBase')
        if internal_db is not None:
            db = internal_db.find('ALCMSDataBase')
            return db
    return None

def get_table_from_alcms_db(tag):
    if alcms_db is None:
        return pd.DataFrame()
    node = alcms_db.find(tag)
    if node is None:
        return pd.DataFrame()
    return pd.DataFrame([item.attrib for item in node])



In [ ]:
"""
    This part uses the two functions from the previous cell and creates the dataframes that you will use for the exercise.
"""

alcms_db = get_alcms_database(root)

df_ccrcircuits     = get_table_from_alcms_db('CCRCircuits')
df_segments        = get_table_from_alcms_db('Segments')
df_functions       = get_table_from_alcms_db('Functions')
df_runways         = get_table_from_alcms_db('Runways')
df_lds             = get_table_from_alcms_db('LDs')
df_airports        = get_table_from_alcms_db('Airports')



"""
    It's always good to visualise the dataframes that have been created.
    To double check if they are correct, but also because they might show you some fields that you were previously unaware of.
"""

print("Segments:", df_segments.columns)
print("Functions:", df_functions.columns)
print("LDs:", df_lds.columns)
print("Runways:", df_runways.columns)
print("Airports:", df_airports.columns)
print("Circuits: ", df_ccrcircuits.columns)

In [ ]:
"""
    Here are two more functions, that can be useful for the exercise.
    You don't have to use them! You can also solve the exercises a different way if you wish to.
    And if you want to make and use any other function you can add them to this cell and call them in the next cell where you have to write your code.
"""

def parse_ids(val):
    """
    Parse strings like:
      "3,5,8" -> ["3","5","8"]
      "1-4"   -> ["1","2","3","4"]
      ""/NaN  -> []
    """
    if val is None or (isinstance(val, float) and pd.isna(val)) or str(val).strip() == "":
        return []
    s = str(val).strip()

    out = []
    for token in s.split(","):
        token = token.strip()
        if not token:
            continue
        if "-" in token:
            a, b = token.split("-", 1)
            a, b = a.strip(), b.strip()
            if a.isdigit() and b.isdigit():
                out.extend([str(i) for i in range(int(a), int(b) + 1)])
            else:
                out.append(token)
        else:
            out.append(token)
    return out

def filter_by_ids(df, ids, id_col="ID"):
    """Safe .isin() even if df[id_col] is int and ids are str (or vice versa)."""
    ids = {str(x) for x in ids if str(x).strip() != ""}
    if not ids:
        return df.iloc[0:0].copy()
    return df[df[id_col].astype(str).isin(ids)].copy()


In [ ]:
"""
    From the next cell onwards, you will have to write your own code to complete the exercise.
    Below is an example function that can help you fill in the other functions, the template is *almost* identical for all the functions.
    All the information needed has been provided in the previous cells, you can use any chatbots you'd like to help you with the task.

    Complete all the functions that say 'Enter your code here' and run the cell.

"""

# 1) Airport -> Runways
def get_airport_and_runways(df_airports, df_runways, airport_id):
    airport_row = df_airports.loc[df_airports["ID"].astype(str) == str(airport_id)]
    if airport_row.empty:
        raise KeyError(f"Airport ID {airport_id} not found")

    airport = airport_row.iloc[0]
    runway_ids = parse_ids(airport.get("RunwayIDs"))
    df_runways_for_airport = filter_by_ids(df_runways, runway_ids)

    return airport, runway_ids, df_runways_for_airport

In [ ]:
# 2) Runways -> LDs
def get_lds_for_runways(df_lds, df_runways_for_airport):

    # ENTER YOUR CODE HERE

    return ld_ids, df_lds_for_airport

In [ ]:
# 3a) LDs -> Functions
def get_function_ids_from_lds(df_lds_for_airport):

    # ENTER YOUR CODE HERE

    return ld_function_ids

In [ ]:
# 3b) Runways -> Functions (direct)
def get_function_ids_from_runways(df_runways_for_airport):

    # ENTER YOUR CODE HERE

    return runway_function_ids

In [ ]:
# 3) Final Functions for Airport (LD + runway-direct-only)
def get_functions_for_airport(df_functions, ld_function_ids, runway_function_ids):

    # ENTER YOUR CODE HERE

    return function_ids, runway_direct_only, df_functions_for_airport

In [ ]:
# 4) Functions -> Segments
def get_segments_for_functions(df_segments, df_functions_for_airport):

    # ENTER YOUR CODE HERE

    return segment_ids, df_segments_for_airport

In [ ]:
# 5) Segments -> Circuits (IDs)
def get_circuit_ids_for_segments(df_segments_for_airport):

    # ENTER YOUR CODE HERE

    return circuit_ids

In [ ]:
"""
    Run this cell to complete all the functions make the final dataframe, in table format and compare it to the excel sheet.
    If you made the functions correctly you will get a passed output.
"""

airport, runway_ids, df_runways_for_airport = get_airport_and_runways(df_airports, df_runways, airport_id="1")

ld_ids, df_lds_for_airport = get_lds_for_runways(df_lds, df_runways_for_airport)

ld_function_ids = get_function_ids_from_lds(df_lds_for_airport)
runway_function_ids = get_function_ids_from_runways(df_runways_for_airport)

function_ids, runway_direct_only_function_ids, df_functions_for_airport = get_functions_for_airport(
    df_functions, ld_function_ids, runway_function_ids
)

segment_ids, df_segments_for_airport = get_segments_for_functions(df_segments, df_functions_for_airport)

circuit_ids = get_circuit_ids_for_segments(df_segments_for_airport)

df_chain = fn.build_airport_chain(df_airports, df_runways, df_lds, df_functions, df_segments, df_ccrcircuits)

res = fn.check_chain_against_excel(df_chain, "Queenstown.xlsx")